# Load Libraries

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from keras.models import model_from_json
import pandas as pd
from hdfs import InsecureClient
import mlflow
import os
import shutil

# Connect to Hadoop

In [ ]:
client_hdfs = InsecureClient('http://awscdh6-ma.sap.local:9870', user='dr.who')

In [ ]:
client_hdfs.list('/tmp/tbr/BARMER/DSP')

# Load Model from Hadoop

Model Structure:

In [ ]:
with client_hdfs.read('/tmp/tbr/BARMER/DSP/model/model_structure.json', encoding='utf-8') as reader:
    loaded_model_json = reader.read()
    reader.close()

model = model_from_json(loaded_model_json)

In [ ]:
loaded_model_json[:100]

Model Weights:

In [ ]:
path = "/tmp/tbr/BARMER/DSP/model/model_weights.h5"
download_path = client_hdfs.download(path, 'download', overwrite=True)
download_path

In [ ]:
model.load_weights(download_path)

Compile model:

In [ ]:
# compile loaded model 
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Delete temp folder

In [ ]:
os.remove("download")

# Load Data (unlabeled) from Hadoop

In [ ]:
# Check Trainings Data
client_hdfs.list('/tmp/tbr/BARMER/DSP/data_unlabeled_predictions/Inertial Signals')

In [ ]:
def load_dataset(prefix):
    # load data and labels
    X = load_dataset_group(prefix)  
   
    # return dataset
    return X

In [ ]:
# load a dataset group
def load_dataset_group(group):
    
    # load all 9 files as a single array
    filenames = list()
    
    # total acceleration
    filenames += ['/Inertial Signals/total_acc_x.txt',
                  '/Inertial Signals/total_acc_y.txt',
                  '/Inertial Signals/total_acc_z.txt']
    
    # body acceleration
    filenames += ['/Inertial Signals/body_acc_x.txt',
                  '/Inertial Signals/body_acc_y.txt',
                  '/Inertial Signals/body_acc_z.txt']
    
    # body gyroscope
    filenames += ['/Inertial Signals/body_gyro_x.txt',
                  '/Inertial Signals/body_gyro_y.txt',
                  '/Inertial Signals/body_gyro_z.txt']
    
    # load input data
    X = load_group(filenames, group)
       
    # return X
    return X

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, group):
    loaded = list()
    
    for name in filenames:

        data = load_file(group+name)
        loaded.append(data)
    
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
    #dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    
    path = '/tmp/tbr/BARMER/DSP/' + filepath
    
    with client_hdfs.read(path, encoding = 'utf-8') as reader:
        dataframe = pd.read_csv(reader, header=None, delim_whitespace=True)   
        
    return dataframe.values

In [ ]:
# load data
X = load_dataset('data_unlabeled_predictions')

In [ ]:
X[0]

In [ ]:
X[0].shape

# Data Preprocessing

# Make Predictions

In [ ]:
def make_prediction(X):
    # define parameters
    verbose = 1
    samples = X.shape[0]
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features   

    
    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    X = X.reshape((samples, time_steps, rows, columns, channels))
        
    # fit network
    class_predictions = model.predict_classes(X, verbose=verbose)    
    
    return class_predictions

In [ ]:
predictions = pd.DataFrame(make_prediction(X), columns=['Predicted Activity Code'])

- 0 WALKING
- 1 WALKING_UPSTAIRS
- 2 WALKING_DOWNSTAIRS
- 3 SITTING
- 4 STANDING
- 5 LAYING

In [ ]:
mapping = {0:"WALKING",
           1:"WALKING_UPSTAIRS",
           2:"WALKING_DOWNSTAIRS",
           3:"SITTING",
           4:"STANDING",
           5:"LAYING"}

In [ ]:
pred_activity_text = [mapping[code] for code in predictions['Predicted Activity Code']]

In [ ]:
predictions['Predicted Activity Text'] = pred_activity_text

In [ ]:
predictions.sample(10).sort_values("Predicted Activity Code")

# Write Predictions to Hadoop

In [ ]:
path = "/tmp/tbr/BARMER/DSP/data_unlabeled_predictions/predictions.csv"

with client_hdfs.write(path, encoding = 'utf-8', overwrite=True) as writer:
    predictions.to_csv(writer)

In [ ]:
client_hdfs.list("/tmp/tbr/BARMER/DSP/data_unlabeled_predictions/")

# Update MLflow on Hadoop

In [ ]:
client_hdfs.download("/tmp/tbr/BARMER/DSP/model/mlruns", "./mlruns", overwrite=True)

In [ ]:
mlflow.set_tracking_uri('./mlruns')
experiment = mlflow.get_experiment_by_name('Activity_Recognition')
experiment.experiment_id

In [ ]:
#mlflow.set_experiment(name='Activity_Recognition')
mlflow.start_run(experiment_id=experiment.experiment_id,
                 run_name='02_make_prediction')
mlflow.log_metric("not available", 0)
mlflow.log_param("model","Keras")
#mlflow.log_artifact("02_make_prediction.ipynb")
#mlflow.log_artifact("02_make_prediction.ipynb")
mlflow.log_artifact("/vrep/vflow/tmp/DSP/EXECUTED/02_make_prediction.ipynb") #Location in Datahub
mlflow.end_run()

In [ ]:
local_path = mlflow.get_tracking_uri()
mlflow.end_run()
local_path

In [ ]:
path = "/tmp/tbr/BARMER/DSP/model/"
_ = client_hdfs.upload(hdfs_path=path, local_path=local_path, overwrite=True)

In [ ]:
client_hdfs.list('/tmp/tbr/BARMER/DSP/model/mlruns')

Delete temp folder

In [ ]:
shutil.rmtree(local_path, ignore_errors=True)